In [1]:
#creating Visualizations of Wage and Cost Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
CostDF = pd.read_csv('StateCostDF.csv',index_col=0)
WageDF = pd.read_csv('StateWageDF.csv',index_col=0)
#salary and price parity
SalPar = pd.read_csv('combined.csv')

In [ ]:
#WageDF.head(5)

In [ ]:
SalPar.rename(columns = {'Standardized Income':'Income v Parity'}, inplace = True)
SalPar.head()

In [ ]:
CostSum = CostDF.where(CostDF['CostType'] != 'Required annual income after taxes').dropna(subset='CostType')
CostSum = CostSum.where(CostSum['CostType'] != 'Required annual income before taxes').dropna(subset='CostType')
CostSum = CostSum.groupby('State').sum()
CostSum.reset_index(inplace=True)
CostSum.head()

In [ ]:
WageTarg = WageDF.where(WageDF['WageType'] == 'Living Wage').dropna(subset='WageType')
WageTarg.drop(['WageType'],axis=1,inplace=True)
for column in WageTarg.columns:
    if column == 'State':
        continue
    else:
        WageTarg[column] = WageTarg[column].apply(lambda x: x * 40 * 52)


In [ ]:
WageTarg.columns

In [ ]:
WagevCost = pd.DataFrame(columns=WageTarg.columns)
for column in WageTarg.columns:
    if column == 'State':
        continue
    else:
        Wage = list(WageTarg[column])
        Cost = list(CostSum[column])
        WvC = []
        if 'BOTH WORKING' in str(column):
            for i in range(len(Wage)):
                WvC.append(2*Wage[i] / Cost[i])
        else:
            for i in range(len(Wage)):
                WvC.append(Wage[i] / Cost[i])
        #print(len(WvC))
        WagevCost[column] = [x for x in WvC]
#
#print(adding)

In [ ]:
WagevCost.drop(['2 ADULTS(1 WORKING) - 0','2 ADULTS(1 WORKING) - 1',
             '2 ADULTS(1 WORKING) - 2','2 ADULTS(1 WORKING) - 3'],axis=1,inplace=True)
WagevCost['State'] = [x for x in WageTarg['State']]
WagevCost.head()
#map this data in a visualization

In [ ]:
WagevCost2 = pd.DataFrame(WagevCost['State'])
WagevCost2['No Children'] = (WagevCost['1 ADULT - 0'] + WagevCost['2 ADULTS(BOTH WORKING) - 0'])/2
WagevCost2['1 Child'] = (WagevCost['1 ADULT - 1'] + WagevCost['2 ADULTS(BOTH WORKING) - 1'])/2
WagevCost2['2 Children'] = (WagevCost['1 ADULT - 2'] + WagevCost['2 ADULTS(BOTH WORKING) - 2'])/2
WagevCost2['3 Children'] = (WagevCost['1 ADULT - 3'] + WagevCost['2 ADULTS(BOTH WORKING) - 3'])/2
WagevCost2['Average'] = (WagevCost2['No Children'] + WagevCost2['1 Child']
                                               + WagevCost2['2 Children'] + WagevCost2['3 Children'])/4
WagevCost2.head(3)

In [ ]:
Top5 = ['Wyoming', 'North Dakota', 'South Dakota', 'Massachusetts']
'''Wyoming: $79,701
North Dakota: $79,363
Connecticut: $77,940
South Dakota: $77,481
Massachusetts: $77,300
'''
Top5CompSci = ['Washington', 'Virginia', 'New York', 'California']
BothTop5 = ['Connecticut']

def coloring(x):
    if x in Top5:
        return 'In Salary v Price Top 5'
    elif x in Top5CompSci:
        return 'Comp Sci Top 5'
    elif x in BothTop5:
        return 'In Both Top 5s'
    else:
        return 'Not in either Top 5'

WagevCost2['Top 5 Best Salary v Price?'] = [coloring(x) for x in WagevCost2['State']]
WagevCost2.head(15)

In [ ]:
chart = plt.figure(figsize=(16,8))

chart = sns.scatterplot(x = 'State',y='Average',
                data = WagevCost2, hue = 'Top 5 Best Salary v Price?',legend=True)

'''
chart = sns.scatterplot(x = 'State',y='No Children',
                data = WagevCost2, hue = 'Top 5 Best Salary v Price?',legend=True)

chart = sns.scatterplot(x = 'State',y='1 Child',
                data = WagevCost2, hue = 'Top 5 Best Salary v Price?',legend=False)

chart = sns.scatterplot(x = 'State',y='2 Children',
                data = WagevCost2, hue = 'Top 5 Best Salary v Price?',legend=False)

chart = sns.scatterplot(x = 'State',y='3 Children',
                data = WagevCost2, hue = 'Top 5 Best Salary v Price?',legend=False)
'''
chart.tick_params(axis='x', labelrotation=90)
plt.ylabel('Average Across All Adults and Children')

In [ ]:
#average the wage and cost data horizontally
#add that to the SalPar dataframe
#map that out

WageTargAvg = WageTarg.copy()
WageTargAvg.drop(['State'],axis=1,inplace=True)
#WageTargAvg.drop(['2 ADULTS(1 WORKING) - 0','2 ADULTS(1 WORKING) - 1',
#             '2 ADULTS(1 WORKING) - 2','2 ADULTS(1 WORKING) - 3'],axis=1,inplace=True)
WageTargAvg['Wage Average'] = WageTargAvg.mean(axis=1)
WageTargAvg['State'] = WageTarg['State']
WageTargAvg.drop(['1 ADULT - 0', '1 ADULT - 1', '1 ADULT - 2', '1 ADULT - 3',
       '2 ADULTS(1 WORKING) - 0', '2 ADULTS(1 WORKING) - 1',
       '2 ADULTS(1 WORKING) - 2', '2 ADULTS(1 WORKING) - 3',
       '2 ADULTS(BOTH WORKING) - 0', '2 ADULTS(BOTH WORKING) - 1',
       '2 ADULTS(BOTH WORKING) - 2', '2 ADULTS(BOTH WORKING) - 3'],axis=1,inplace=True)

CostSumAvg = CostSum.copy()
CostSumAvg.drop(['State'],axis=1,inplace=True)
#CostSumAvg.drop(['2 ADULTS(1 WORKING) - 0','2 ADULTS(1 WORKING) - 1',
#             '2 ADULTS(1 WORKING) - 2','2 ADULTS(1 WORKING) - 3'],axis=1,inplace=True)
CostSumAvg['Cost Average'] = CostSumAvg.mean(axis=1)
CostSumAvg['State'] = CostSum['State']
CostSumAvg.drop(['1 ADULT - 0', '1 ADULT - 1', '1 ADULT - 2', '1 ADULT - 3',
       '2 ADULTS(1 WORKING) - 0', '2 ADULTS(1 WORKING) - 1',
       '2 ADULTS(1 WORKING) - 2', '2 ADULTS(1 WORKING) - 3',
       '2 ADULTS(BOTH WORKING) - 0', '2 ADULTS(BOTH WORKING) - 1',
       '2 ADULTS(BOTH WORKING) - 2', '2 ADULTS(BOTH WORKING) - 3'],axis=1,inplace=True)

CompSci = pd.read_csv('ComputerScience.csv',index_col=0)
CompSci['Typical Annual Salary'] = [x[1:] for x in CompSci['Typical Annual Salary']]
CompSci['Typical Annual Salary'] = [int(x.replace(',','')) for x in CompSci['Typical Annual Salary']]

SalPar.set_index('State',inplace=True)
WageTargAvg.set_index('State',inplace=True)
CostSumAvg.set_index('State',inplace=True)
CompSci.set_index('State',inplace=True)
SalPar = SalPar.join(WageTargAvg)
SalPar = SalPar.join(CostSumAvg)
SalPar = SalPar.join(CompSci)
SalPar = SalPar.dropna()
SalPar.reset_index(inplace=True)
SalPar.head(5)

In [ ]:
chart = plt.figure(figsize=(16,8))

chart1 = sns.scatterplot(x = 'State',y='Income v Parity',
                data = SalPar, legend=True)

chart2 = sns.scatterplot(x = 'State',y='Wage Average',
                data = SalPar, legend=True)

chart3 = sns.scatterplot(x = 'State',y='Cost Average',
                data = SalPar, legend=True)

#chart4 = sns.scatterplot(x = 'State',y='Typical Annual Salary',
#                data = SalPar, legend=True)

chart1.tick_params(axis='x', labelrotation=90)
#plt.legend([chart1, chart2, chart3], ['Income v Parity', 'Wage Average', 'Cost Average'])